In [1]:
# Computing
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.neighbors import KernelDensity
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
import time

import matplotlib
matplotlib.use("cairo")
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib import font_manager
from mpl_toolkits.axes_grid1 import make_axes_locatable
from bootplot import bootplot
from bootplot.backend.matplotlib import plot_to_array, close_figure
from pywaffle import Waffle
from pysankey2 import Sankey
import networkx as nx
from matplotlib.colors import ListedColormap

# Image processing
import matplotlib.image as mpimg
import cv2
from PIL import Image

# Display tools
from IPython.display import display

In [2]:
# Combine and show two images one next to each other.
def show_images(image_paths, output_path="", above=False):
    if len(image_paths) > 1:
        images = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in image_paths]
        above = 0 if above else 1
        combined = np.concatenate(images, axis=above)
        Image.fromarray(combined).save(output_path)
    else:
        output_path=image_paths[0]
    with Image.open(output_path) as img:
        display(img)

In [3]:
dpi = 300
plt.rcParams['figure.dpi'] = dpi
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['font.size'] = 12

In [4]:
# Read the csv file and create pandas dataframe
csv_file_path = 'penguins.csv'
df = pd.read_csv(csv_file_path)
penguins = df.dropna()
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


In [5]:
penguins_bill_length = penguins[['species', 'bill_length_mm']]
penguins_bill_length_depth = penguins[['species','bill_length_mm', 'bill_depth_mm']].values

In [6]:
def measure_time(func, n):
    times = []
    for _ in range(n):
        start = time.perf_counter()
        func()
        times.append(time.perf_counter() - start)
    return times

In [7]:
def timer_mean():
    def mean_estimate(data_subset, data_full, ax):
        colors = dict(zip(['Adelie', 'Chinstrap', 'Gentoo'], ['#FF8C00', '#A020F0', '#008B8B']))
        means = data_subset.groupby('species')['bill_length_mm'].mean().reindex(colors)
        for i, (species, mean) in enumerate(means.items()):
            if pd.notna(mean):
                ax.scatter(mean, i, s=10, c=colors[species], antialiased=False,marker='s',zorder=10)
        ax.set(
            xlim=(33, 55), xticks=np.arange(33, 58, 5), xlabel='Bill Length',
            ylim=(-0.5, 2.5), yticks=[0, 1, 2], ylabel=''
        )
        ax.set_yticklabels(['Adelie', 'Chinstrap', 'Gentoo'], ha='right')
        
        ax.grid(axis='both', linewidth=0.5, color='#E8E8E8', zorder=-50)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        ax.get_figure().set_constrained_layout(True)

    np.random.seed(15)
    mat = bootplot(mean_estimate,
            penguins_bill_length,
            m=39,
            output_image_path=f'mean_bootplot.png',
            output_size_px = (dpi*3, dpi*1.5),
            verbose=False)

In [8]:
def timer_mean_sns():
    fig, ax = plt.subplots(figsize=(3,1.5), dpi=dpi, layout="constrained")

    species_order = ['Adelie', 'Chinstrap', 'Gentoo']
    palette = dict(zip(species_order, ['#FF8C00', '#A020F0', '#008B8B']))
    
    sns.pointplot(
        data=penguins_bill_length,
        y='species',
        x='bill_length_mm',
        hue='species',
        estimator=np.mean,
        order=species_order,
        hue_order=species_order,
        palette=palette,
        errorbar=('ci', 95),
        n_boot=5000,
        err_kws={'linewidth': 4},
        linestyle='none',
        markers='',
        dodge=False,
        capsize=0, 
        seed=15
    )
    
    ax.set(
        xlim=(33, 55), xticks=np.arange(33, 58, 5), xlabel='Bill Length',
        ylim=(-0.5, 2.5), yticks=[0, 1, 2], ylabel=''
    )
    ax.set_yticklabels(['Adelie', 'Chinstrap', 'Gentoo'], ha='right')
        
    ax.grid(axis='both', linewidth=0.5, color='#E8E8E8', zorder=-50)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.savefig(f'mean_sns.png', format='png')
    plt.close()

In [9]:
bootplot_time_mean = measure_time(timer_mean, 39)
sns_time_mean = measure_time(timer_mean_sns, 39)

In [10]:
def timer_poly_line_bootplot():
    mono_font = font_manager.FontProperties(fname='/System/Library/Fonts/SFNSMono.ttf', size=15)
    colors = ['#FF8C00', '#A020F0', '#008B8B']
    order=0
    deg=4
    ci=95
    def lr_poly_line(data_subset, data_full, ax, order=0):
        ax.grid(True, linewidth=0.5, color='#E8E8E8', zorder=-100)
        ax.set_axisbelow(True)
        for i, species in enumerate(np.unique(data_full[:, 0])):
            subset = data_full[data_full[:, 0] == species]
            ax.scatter(subset[:, 1], subset[:, 2], c=colors[i], s=20, alpha=0.5,
                       edgecolor='grey', linewidths=0.75, marker='o', label=species, zorder=100)
    
        x_min, x_max = data_full[:, 1].min(), data_full[:, 1].max()
        y_min, y_max = data_full[:, 2].min(), data_full[:, 2].max()
        
        def plot_model(x, y, model, transform=None, color='red', lw=1, z=50, annotate_rmse=False, rmse_pos='left'):
            x_range = np.linspace(x_min, x_max, 200).reshape(-1, 1)
            X_train = transform(x) if transform else x
            X_plot = transform(x_range) if transform else x_range
            model.fit(X_train, y)
            ax.plot(x_range.ravel(), model.predict(X_plot), c=color, lw=lw, antialiased=False, zorder=z)
            if annotate_rmse:
                rmse = np.sqrt(np.mean((y - model.predict(X_train)) ** 2))
                ax.text(
                    x=x_min + 0.7 if rmse_pos == 'left' else x_max - 8,
                    y=(2 * y_max - 1) / 2,
                    s=f'RMSE: {rmse:.2f}',
                    fontsize=8,
                    ha='left', va='top',
                    color='#c44e52' if rmse_pos == 'left' else '#264d73',
                    fontproperties=mono_font,
                    bbox=dict(facecolor='white', edgecolor='#E8E8E8',
                              boxstyle='round,pad=0.2', linewidth=0.3, alpha=1),
                    zorder=1000
                )
        
        plot_model(data_subset[:, 1:2], data_subset[:, 2], LinearRegression(),
                   color='#c44e52', lw=1, z=order % 4 + 500, annotate_rmse=True, rmse_pos='left')
    
        plot_model(data_full[:, 1:2], data_full[:, 2], LinearRegression(),
                   color='#c44e52', lw=1, z=(order + 1) % 4 + 500)
    
        poly = PolynomialFeatures(degree=deg, include_bias=False).fit(data_subset[:, 1:2])
        plot_model(data_subset[:, 1:2], data_subset[:, 2], LinearRegression(),
                   transform=poly.transform, color='#264d73', lw=1,
                   z=(order + 2) % 4 + 500, annotate_rmse=True, rmse_pos='right')
    
        poly_full = PolynomialFeatures(degree=deg, include_bias=False).fit(data_full[:, 1:2])
        plot_model(data_full[:, 1:2], data_full[:, 2], LinearRegression(),
                   transform=poly_full.transform, color='#264d73', lw=1, z=(order + 3) % 4 + 500)
    
        
        ax.set(xlabel='Bill Length', ylabel='Bill Depth',
               xlim=(x_min, x_max), ylim=(y_min, y_max),
               xticks=np.arange(np.ceil(x_min), np.floor(x_max) + 1, 5),
               yticks=np.arange(np.ceil(y_min), np.floor(y_max) + 1, 2))
        ax.get_figure().set_constrained_layout(True)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    np.random.seed(15)
    mat = bootplot(
            lr_poly_line,
            penguins_bill_length_depth,
            m=39,
            output_image_path=f'poly_line_bootplot.png',
            output_size_px = (dpi*3, dpi*2.5),
            verbose=False)

In [11]:
def timer_poly_line_sns():
    mono_font = font_manager.FontProperties(fname='/System/Library/Fonts/SFNSMono.ttf', size=15)
    colors = ['#FF8C00', '#A020F0', '#008B8B']
    order=0
    deg=4
    ci=95
    fig, ax = plt.subplots(figsize=(3,2.5), dpi=dpi, layout="constrained")
    ax.grid(True, linewidth=0.5, color='#E8E8E8', zorder=-100)
    ax.set_axisbelow(True)
    
    
    for i, species in enumerate(np.unique(penguins_bill_length_depth[:, 0])):
        subset = penguins_bill_length_depth[penguins_bill_length_depth[:, 0] == species]
        ax.scatter(subset[:, 1], subset[:, 2], c=colors[i], alpha=0.5, linewidths=0.75, s=20, edgecolor='grey',
                   zorder=100, marker='o')
    
    
    X = penguins_bill_length_depth[:, 1].astype(float).reshape(-1, 1)
    y = penguins_bill_length_depth[:, 2].astype(float)
    
       
    def add_rmse(model, Xp, label, x_pos):
        rmse = np.sqrt(np.mean((y - model.predict(Xp)) ** 2))
        ax.text(x_pos, (2 * y.max() - 1) / 2, f'RMSE: {rmse:.2f}', fontsize=8, ha='left', va='top', color=label,
                fontproperties=mono_font, bbox=dict(facecolor='white', edgecolor='#E8E8E8',
                boxstyle='round,pad=0.2', linewidth=0.3, alpha=1), zorder=1000)
        
    
    sns.regplot(x=X.ravel(), y=y, scatter=False, ci=ci,
                line_kws={'color': '#c44e52', 'linewidth': 1, 'zorder': 500}, seed=15)
    lm = LinearRegression().fit(X, y)
    add_rmse(lm, X, '#c44e52', X.min() + 0.7)
        
    
    sns.regplot(x=X.ravel(), y=y, scatter=False, order=deg, ci=ci,
                line_kws={'color': '#264d73', 'linewidth': 1, 'zorder': 500}, seed=15)
    poly = PolynomialFeatures(degree=deg, include_bias=False)
    X_poly = poly.fit_transform(X)
    lm_poly = LinearRegression().fit(X_poly, y)
    add_rmse(lm_poly, X_poly, '#264d73', X.max() - 8)
        
    
    ax.set(xlabel='Bill Length', ylabel='Bill Depth',
           xlim=(X.min(), X.max()), ylim=(y.min(), y.max()),
           xticks=np.arange(np.ceil(X.min()), np.floor(X.max()) + 1, 5),
           yticks=np.arange(np.ceil(y.min()), np.floor(y.max()) + 1, 2))
    ax.spines[['top', 'right']].set_visible(False)
        
    plt.savefig(f'poly_line_sns.png', format='png')
    plt.close(fig)

In [12]:
bootplot_time_poly_line = measure_time(timer_poly_line_bootplot, 39)
sns_time_poly_line = measure_time(timer_poly_line_sns, 39)

In [13]:
def calculate_results(times):
    n = len(times)
    mean = np.mean(times)
    std = np.std(times, ddof=1) 
    se = std / np.sqrt(n)
    
    t_crit = stats.t.ppf(0.975, df=n-1)
    margin = t_crit * se

    return f"{mean:.6f} ± {margin:.6f}"

In [14]:
print(f"Bootplot mean: {calculate_results(bootplot_time_mean)}")
print(f"Sns mean: {calculate_results(sns_time_mean)}")
print(f"Bootplot linear and polynomial regression: {calculate_results(bootplot_time_poly_line)}")
print(f"Sns linear and polynomial regression: {calculate_results(sns_time_poly_line)}")

Bootplot mean: 1.051745 ± 0.016226
Sns mean: 0.094142 ± 0.003891
Bootplot linear and polynomial regression: 2.103895 ± 0.015650
Sns linear and polynomial regression: 0.123690 ± 0.000657
